In [13]:
########################################################### IMPORTS
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from gpflow.utilities import print_summary, set_trainable
import gpflow
import tensorflow as tf
import tensorflow_probability as tfp

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import sys
sys.path.append('../src')
from data_preparation import get_birth_data, plot_data, separate_data

########################################################### DATA
data = get_birth_data()
un_x, un_y = separate_data(data)
un_x_w, un_y_w = separate_data(data, weekdays=True)
x, y = separate_data(data, normalised=True)
x_w, y_w = separate_data(data, normalised=True, weekdays=True)

In [11]:
# g = np.array(range(y.shape[0]))
# g = np.reshape(g, (g.shape[0],1))
# yg = np.hstack([y.numpy(), g])

In [19]:
x_reduced, y_reduced = x[:100], y[:100]

hetero_gp_likelihood = gpflow.likelihoods.HeteroskedasticTFPConditional(
    distribution_class=tfp.distributions.Normal,  # Gaussian Likelihood
    scale_transform=tfp.bijectors.Exp(),  # Exponential Transform
)
print(f"Likelihood's expected latent_dim: {hetero_gp_likelihood.latent_dim}")
hetero_gp_kernel = gpflow.kernels.SeparateIndependent(
    [
        gpflow.kernels.SquaredExponential(),  # This is k1, the kernel of f1
        gpflow.kernels.SquaredExponential(),  # this is k2, the kernel of f2
    ]
)
#Inducing points need to be initialized for using Stochastic Variational Gaussian Process Model
M = 10  # Number of inducing variables for each f_i

# Initial inducing points position Z
Z = x.copy() #np.linspace(x.numpy().min(), x.numpy().max(), M)[:, None]  # Z must be of shape [M, 1]

inducing_variable = gpflow.inducing_variables.SeparateIndependentInducingVariables(
    [
        gpflow.inducing_variables.InducingPoints(Z),  # This is U1 = f1(Z1)
        gpflow.inducing_variables.InducingPoints(Z),  # This is U2 = f2(Z2)
    ]
)
hetero_gp_mdl = gpflow.models.VGP(#SVGP
    data=(x,y),
    kernel=hetero_gp_kernel ,
    likelihood=hetero_gp_likelihood,
    #inducing_variable=inducing_variable,
    num_latent_gps=hetero_gp_likelihood.latent_dim,
)
hetero_gp_mdl

2022-07-08 19:18:16.118973: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.1KiB (rounded to 3328)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-07-08 19:18:16.119027: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-07-08 19:18:16.119049: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 55, Chunks in use: 55. 13.8KiB allocated for chunks. 13.8KiB in use in bin. 2.2KiB client-requested in use in bin.
2022-07-08 19:18:16.119063: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-07-08 19:18:16.119077: I tensorflow/core/common_runt

ResourceExhaustedError: OOM when allocating tensor with shape[20,20] and type double on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill]

 I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 7f2e54066d00 of size 256 next 21
2022-07-08 19:18:16.119642: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 7f2e54066e00 of size 256 next 28
2022-07-08 19:18:16.119650: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 7f2e54066f00 of size 256 next 30
2022-07-08 19:18:16.119659: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 7f2e54067000 of size 12544 next 27
2022-07-08 19:18:16.119668: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 7f2e5406a100 of size 6144 next 32
2022-07-08 19:18:16.119676: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 7f2e5406b900 of size 256 next 19
2022-07-08 19:18:16.119684: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 7f2e5406ba00 of size 256 next 65
2022-07-08 19:18:16.119692: I tensorflow/core/common_runtime/bfc_allocator.cc:1066] InUse at 7f2e5406bb00 of size 256 next 61
2022-07-08 19:18

In [12]:
# likelihood = gpflow.likelihoods.SwitchedLikelihood(
#     [gpflow.likelihoods.Gaussian(variance=1.0), gpflow.likelihoods.Gaussian(variance=1.0)]
# )

In [22]:
kernel = gpflow.kernels.RBF()
model = gpflow.models.GPR((x, yg), kernel=kernel)


In [23]:
set_trainable(model.q_mu, False)
set_trainable(model.q_sqrt, False)

AttributeError: 'GPR' object has no attribute 'q_mu'

In [19]:
from gpflow.ci_utils import ci_niter
opt1 = gpflow.optimizers.NaturalGradient(gamma=1.0)
opt2 = tf.optimizers.Adam()
for _ in range(ci_niter(1000)):
    opt1.minimize(model.training_loss, [(model.q_mu, model.q_sqrt)])
    opt2.minimize(model.training_loss, model.trainable_variables)

2022-06-20 19:29:49.124532: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 203.59MiB (rounded to 213481472)requested by op ReverseV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-06-20 19:29:49.124593: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-06-20 19:29:49.124618: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 66, Chunks in use: 66. 16.5KiB allocated for chunks. 16.5KiB in use in bin. 1.4KiB client-requested in use in bin.
2022-06-20 19:29:49.124632: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-06-20 19:29:49.124645: I tensorflow/cor

ResourceExhaustedError: OOM when allocating tensor with shape[1,26685165] and type double on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ReverseV2]